<a href="https://colab.research.google.com/github/DelWow/RippenApril2025/blob/main/Fragmentstein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Setup process
! pip install "numpy==1.26.4"
!pip install fragmenstein
!pip install rdkit
!pip install mmpdb


In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, rdmolfiles
from fragmenstein import Frankenstein

In [29]:
!git branch


In [ ]:
!mmpdb --help

Usage: mmpdb [OPTIONS] COMMAND [ARGS]...

  Matched-molecular pair database loader

Options:
  -q, --quiet  Do not show progress or status information
  --version    Show the version and exit.
  --help       Show this message and exit.

Commands:

 Matched molecular pair generation commands (see 'help-analysis'):
  fragment       Fragment SMILES file structures on rotatable bonds
  smifrag        Fragment a single SMILES string
  index          Index fragments and find matched molecular pairs
  predict        Predict the effect of a structural transformation
  transform      Transform a structure
  rgroup2smarts  Convert an R-group file into a SMARTS which matches all of
                 the SMILES
  generate       Apply database transforms to a molecule

 Distributed generation commands (see 'help-distributed'):
  smi_split         Split the SMILES file 'FILE' into smaller files
  fragdb_constants  List constants fragdb DATABASEs and their frequencies
  fragdb_partition  Partition fra

In [ ]:
!mmpdb search "attachment_points == 2" test_data.fragdb > chosen_frags.smi

Usage: mmpdb [OPTIONS] COMMAND [ARGS]...
Try 'mmpdb --help' for help.

Error: No such command 'search'.


In [ ]:
from fragmenstein import Monster
from rdkit import Chem


# Load fragments from files (must be aligned in 3D)
mol1 = Chem.MolFromMolFile('fragment1.mol', removeHs=False)
mol2 = Chem.MolFromMolFile('fragment2.mol', removeHs=False)

# Set names (Fragmenstein uses them to track)
mol1.SetProp('_Name', 'frag1')
mol2.SetProp('_Name', 'frag2')

# Initialize Fragmenstein Monster
monster = Monster(
    receptor_path='/content/1SJP.pdb',  # Provide your receptor PDB file
    idealize=True,
    name='linked_molecule'
)

# Assign the fragments (must have 3D coordinates)
monster.frags = [mol1, mol2]

# Combine them using Fragmenstein
merged_mol = monster.combine()

# Save the result
Chem.MolToMolFile(merged_mol, 'linked_output.mol', includeStereo=True)

# Also print SMILES for sanity
print("Linked Molecule SMILES:", Chem.MolToSmiles(merged_mol))

TypeError: _MonsterBase.__init__() got an unexpected keyword argument 'receptor_path'

In [ ]:
from fragmenstein import Victor
from rdkit import Chem

# Define the input fragments as SMILES strings
fragment1 = Chem.MolFromSmiles("CCO")
fragment2 = Chem.MolFromSmiles("CCN")

# Create a Victor instance for fragment merging
victor = Victor(hits=[fragment1, fragment2])

# Merge the fragments
merged_molecule = victor.combine()

# Convert the merged molecule back to SMILES
merged_smiles = Chem.MolToSmiles(merged_molecule)
print(f"Merged molecule: {merged_smiles}")


In [ ]:
from fragmenstein import Victor
from rdkit import Chem
from rdkit.Chem import AllChem

# Load fragments as SMILES strings
fragment1 = Chem.MolFromSmiles("CCO")  # Example fragment 1
fragment2 = Chem.MolFromSmiles("CCN")  # Example fragment 2

# Assign 3D coordinates (normally from docking results or a PDB)
AllChem.EmbedMolecule(fragment1)
AllChem.EmbedMolecule(fragment2)

# Load the protein-ligand complex (PDB) for context
protein_pdb = "protein_structure.pdb"  # Replace with your actual PDB file

# Initialize Victor with the binding site and fragment information
victor = Victor(
    hits=[fragment1, fragment2],  # Fragments to merge
    pdb_filename=protein_pdb,  # Reference protein structure
    covalent=False,  # If fragments should be merged covalently
    discount_flipper=True  # Reduce unnecessary flipping of fragments
)

# Merge the fragments and generate the final molecule
merged_molecule = victor.combine()

# Optimize the merged molecule to fit the binding site
victor.positioned_mol  # This is the final molecule with adjusted positioning

# Save output as an SDF file for visualization
w = Chem.SDWriter("merged_molecule.sdf")
w.write(victor.positioned_mol)
w.close()

# Print the final SMILES representation
final_smiles = Chem.MolToSmiles(victor.positioned_mol)
print(f"Final merged molecule: {final_smiles}")